__Federated Learning__



In [39]:
import random

import sklearn.metrics
import torch
from torch import Tensor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# from models import LogisticRegressionModel
# from Tensor import Tensor

device = "cuda" if torch.cuda.is_available() else "cpu"

__Data Preprocessing__

In [44]:
device = torch.device(f'cuda:{torch.cuda.current_device()}') if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
torch.set_default_dtype(torch.float32)

In [45]:
df = pd.read_csv("mushrooms.csv")
df = pd.get_dummies(df, drop_first=True).astype(float)
y = df.class_p
X = df.drop('class_p', axis=1)
X_train, X_test, y_train, y_test = (
    Tensor(i.to_numpy()) for i in train_test_split(X, y, test_size=0.33, random_state=239)
)




In [46]:
class LogisticRegressionModel:

    """
    For outer optimizer working cycle should be:
    For k in 1.....N:
        y_pred = [model.predict(x_i) for random x_i in X]
        loss = model.calculateLoss(y_pred, y_true)
        dL_dW, dL_db = model.calculateGradients(X, y_pred=y_pred, y_true=y_true)

        ### Compute the update for the weights

        model.scaleParameters(scale_factor)  (optionally)
        model.updateParameters(delta_W, delta_B)
    """
    def __init__(self, params_amount: int):
        self.weights = Tensor(np.zeros(params_amount))
        self.bias = Tensor([0])
        self.params_amount = params_amount

    def calculateZ(self, X: Tensor) -> Tensor:
        return self.weights @ X.T + self.bias

    def predict(self, x: Tensor) -> Tensor:
        z = self.calculateZ(x)
        if z.item() >= 0:
            return Tensor(1 / (1 + torch.exp(-1 * z)))
        return torch.exp(z) / (1 + torch.exp(z))

    def calculateLoss(self, y_pred: Tensor, y_true: Tensor) -> Tensor:
        y_zero_loss = torch.dot(y_true, torch.log(y_pred + 1e-9))
        y_one_loss = torch.dot(
            Tensor(y_true.size()[0]) - y_true,
            torch.log(Tensor(np.ones(y_pred.size()[0])) - y_pred)
        )
        return -1 * (y_zero_loss + y_one_loss) / self.params_amount

    @staticmethod
    def calculateGradients(X: Tensor, y_pred: Tensor, y_true: Tensor) -> tuple[Tensor, Tensor]:
        difference = y_pred - y_true
        dL_db = Tensor(torch.mean(difference))
        JW = X.T @ difference
        dL_dW = Tensor([torch.mean(grad) for grad in JW])   # VERY UNCERTAIN ABOUT THIS IN MATHEMATICAL SENSE
        return dL_dW, dL_db

    def updateParameters(self, deltaW: Tensor, deltaB: Tensor) -> None:
        self.weights = self.weights + deltaW
        self.bias = self.bias + deltaB

    def scaleParameters(self, scale_factor: float) -> None:
        self.weights = Tensor(scale_factor * self.weights)
        self.bias = Tensor(scale_factor * self.bias)

    def predictClass(self, X: Tensor) -> int:
        return 1 if self.predict(X).item() >= 0.5 else 0
    


class NaiveGradientDescent:
    
    def __init__(self, alpha: float, iterations: int, batch_size: int):
        self.alpha = alpha
        self.iterations = iterations
        self.batch_size = batch_size
        
    def train(self, X_train, y_train):
        self.model = LogisticRegressionModel(X_train.size()[1])
        for k in range(self.iterations):
            y_pred = torch.zeros(self.batch_size)
            y_true= torch.zeros(self.batch_size)
            batch = torch.zeros((self.batch_size, X_train.size()[1]))
            for i in range(self.batch_size):
                random_index = random.randint(0, y_train.size()[0] - 1)
                x = X_train[random_index]
                batch[i] = x
                y_pred[i] = self.model.predict(x).item()
                y_true[i] = y_train[i].item()
            print(self.model.calculateLoss(y_pred, y_true))
            dL_dW, dL_db = self.model.calculateGradients(batch, y_pred, y_true)
            self.model.updateParameters(-1 * self.alpha * dL_dW, -1 * self.alpha * dL_db)

GD = NaiveGradientDescent(0.01, 1000, 10)
GD.train(X_train, y_train)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!